In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from citipy import citipy

# Range of latitudes and longitudes
lat_range = (64, 20)
lng_range = (-154, -65)

# Import API key
from api_keys import weather_key

In [32]:
weather_data_df = pd.read_csv("data/noaa_storm_data.csv")
weather_data_df.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE,EPISODE_NARRATIVE,ABSOLUTE_ROWNUMBER
0,866322,CARTERET CO.,TRUTTNEYS LNDG,11/1/2019,15,Tornado,,EF0,0,0,...,1,NNW,TRUTTNEYS LNDG,34.7394,-77.1094,34.7411,-77.1053,A brief tornado occurred near the intersection...,A strong cold front moved across Eastern North...,1
1,859389,MARTIN CO.,WAVELAND,11/5/2019,1620,Tornado,,EF0,0,0,...,2,ENE,WAVELAND,27.2140,-80.1729,27.2123,-80.1766,Martin County Beach Patrol reported a waterspo...,A strong thunderstorm produced a waterspout ju...,2
2,864077,MOBILE CO.,PRICHARD,11/7/2019,1335,Tornado,,EF0,0,0,...,0,SSE,PRICHARD,30.7248,-88.0548,30.7260,-88.0486,The storm survey team surveyed the areas aroun...,Thunderstorms moved across the area during the...,3
3,857862,FRANKLIN PAR.,BASKIN,11/26/2019,1803,Tornado,,EF2,1,1,...,4,NE,BASKIN,32.2351,-91.7783,32.3184,-91.7095,This tornado began on Roger Miller Road as it ...,"Warm, moist air was in place across northeast ...",4
4,857944,CLAIBORNE PAR.,MAHAN,11/26/2019,1820,Tornado,,EF1,0,0,...,2,NNE,MAHAN,32.9017,-93.0423,32.9076,-93.0343,An EF-1 tornado with estimated maximum winds a...,A deep southwesterly flow aloft ahead of a lar...,5


In [33]:
gmaps.configure(api_key = weather_key)

In [34]:
tornado_df = weather_data_df.loc[:,["BEGIN_LOCATION","BEGIN_DATE", "EVENT_TYPE", "TOR_F_SCALE", "DEATHS_DIRECT", "END_LOCATION"]]
tornado_df

,BEGIN_LOCATION,BEGIN_DATE,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,END_LOCATION
0,TRUTTNEYS LNDG,11/1/2019,Tornado,EF0,0,TRUTTNEYS LNDG
1,WAVELAND,11/5/2019,Tornado,EF0,0,WAVELAND
2,PRICHARD,11/7/2019,Tornado,EF0,0,PRICHARD
3,BASKIN,11/26/2019,Tornado,EF2,1,BASKIN
4,MAHAN,11/26/2019,Tornado,EF1,0,MAHAN
...,...,...,...,...,...,...
495,LAMONT,4/13/2020,Tornado,EF0,0,LAMONT
496,RED OAK,4/13/2020,Tornado,EF0,0,RED OAK
497,COLLIER,4/13/2020,Tornado,EF3,0,BRIGHTON MILLS
498,ROBERTS XRDS,4/13/2020,Tornado,EF0,0,ROBERTS XRDS


In [35]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

528

In [36]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=04c41275fea3e8291170fd88095dde0f"
# List of city data
city_data = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for x, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (x % 50 == 0 and x >= 50):
        set_count += 1
        record_count = 0
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]
        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | fortuna
Processing Record 2 of Set 1 | liniere
City not found. Skipping...
Processing Record 3 of Set 1 | kodiak
Processing Record 4 of Set 1 | pacific grove
Processing Record 5 of Set 1 | norman wells
Processing Record 6 of Set 1 | matagami
Processing Record 7 of Set 1 | rockland
Processing Record 8 of Set 1 | sept-iles
Processing Record 9 of Set 1 | kahului
Processing Record 10 of Set 1 | guerrero negro
Processing Record 11 of Set 1 | blythe
Processing Record 12 of Set 1 | hamilton
Processing Record 13 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 14 of Set 1 | hilo
Processing Record 15 of Set 1 | coos bay
Processing Record 16 of Set 1 | cockburn town
Processing Record 17 of Set 1 | salisbury
Processing Record 18 of Set 1 | virginia beach
Processing Record 19 of Set 1 | lompoc
Processing Record 20 of Set 1 | constitucion
Processing Record 21 of Set 1 | iqaluit
Processing

Processing Record 42 of Set 4 | lake cowichan
Processing Record 43 of Set 4 | logansport
Processing Record 44 of Set 4 | abilene
Processing Record 45 of Set 4 | nantucket
Processing Record 46 of Set 4 | high level
Processing Record 47 of Set 4 | quesnel
Processing Record 48 of Set 4 | grass valley
Processing Record 49 of Set 4 | dzilam gonzalez
Processing Record 0 of Set 5 | viesca
Processing Record 1 of Set 5 | progreso
Processing Record 2 of Set 5 | ada
Processing Record 3 of Set 5 | rancho palos verdes
Processing Record 4 of Set 5 | pell city
Processing Record 5 of Set 5 | oskaloosa
Processing Record 6 of Set 5 | spirit river
Processing Record 7 of Set 5 | two rivers
Processing Record 8 of Set 5 | valentin gomez farias
Processing Record 9 of Set 5 | port hueneme
Processing Record 10 of Set 5 | carteret
Processing Record 11 of Set 5 | great falls
Processing Record 12 of Set 5 | escanaba
Processing Record 13 of Set 5 | jacksonville
Processing Record 14 of Set 5 | storm lake
Processing

Processing Record 37 of Set 8 | lovington
Processing Record 38 of Set 8 | moorhead
Processing Record 39 of Set 8 | carlsbad
Processing Record 40 of Set 8 | arcata
Processing Record 41 of Set 8 | minnedosa
Processing Record 42 of Set 8 | plainview
Processing Record 43 of Set 8 | clifton
Processing Record 44 of Set 8 | spring hill
Processing Record 45 of Set 8 | emporia
Processing Record 46 of Set 8 | davidson
Processing Record 47 of Set 8 | anahuac
Processing Record 48 of Set 8 | chalchihuites
Processing Record 49 of Set 8 | huron
Processing Record 0 of Set 9 | campbellsville
Processing Record 1 of Set 9 | prince george
Processing Record 2 of Set 9 | orange cove
Processing Record 3 of Set 9 | valleyview
Processing Record 4 of Set 9 | valley city
Processing Record 5 of Set 9 | newberry
Processing Record 6 of Set 9 | deloraine
Processing Record 7 of Set 9 | valley
Processing Record 8 of Set 9 | west wendover
Processing Record 9 of Set 9 | varadero
Processing Record 10 of Set 9 | roanoke r

In [37]:
city_data_pd = pd.DataFrame(city_data)
city_data_pd.count()


City          513
Lat           513
Lng           513
Max Temp      513
Humidity      513
Cloudiness    513
Wind Speed    513
Country       513
Date          513
dtype: int64

In [38]:
city_data_pd.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,fortuna,40.5982,-124.1573,46.99,87,75,16.11,US,1615339375
1,kodiak,57.7900,-152.4072,39.20,81,90,8.05,US,1615339222
2,pacific grove,36.6177,-121.9166,55.40,54,1,8.05,US,1615339341
3,norman wells,65.2820,-126.8329,5.00,61,1,9.22,CA,1615339376
4,matagami,49.7502,-77.6328,39.20,52,90,5.75,CA,1615339376


In [40]:
america_city = city_data_pd[(city_data_pd["Country"]=="US")]
america_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,fortuna,40.5982,-124.1573,46.99,87,75,16.11,US,1615339375
1,kodiak,57.7900,-152.4072,39.20,81,90,8.05,US,1615339222
2,pacific grove,36.6177,-121.9166,55.40,54,1,8.05,US,1615339341
5,rockland,41.1668,-74.0496,46.00,75,0,5.46,US,1615339376
7,kahului,20.8947,-156.4700,71.60,83,90,3.44,US,1615339344
...,...,...,...,...,...,...,...,...,...
499,natchez,31.5604,-91.4032,66.20,67,40,4.61,US,1615339524
504,wahpeton,46.2652,-96.6059,46.40,76,1,17.27,US,1615339525
506,portage,41.1667,-81.1998,53.01,43,1,5.66,US,1615339526
507,belle fourche,44.6714,-103.8522,42.01,60,1,8.05,US,1615339526
